In [ ]:
import itertools
import datetime
import time
import glob
import random
import sys
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

from tqdm import tqdm
import pickle
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import json as js
from scipy.signal import find_peaks

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    cuda = True
    print('Using: ' + str(torch.cuda.get_device_name(device)))
else:
    cuda = False
    print('Using: CPU')

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [ ]:
from t2tganmodel import *
from callablefunc import *

In [ ]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 6
num_rounds = 10
epochs = 5
batch_size = 32

In [ ]:
def save_params(params, filename):
    json = js.dumps(params)
    f = open(filename+'/parameters.json','w')
    f.write(json)
    f.close()

In [ ]:
DATASETPATH = "dataset/Samples"

def load_data(seq_len=1000, num_clients=20, datasetpath=DATASETPATH):
    files = os.listdir(datasetpath)
    files = [f for f in files if f.endswith('.csv')]

    # sort the files
    files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    files = [os.path.join(DATASETPATH, f) for f in files]

    # load the data and combine them
    for i, f in enumerate(files):
        # load data
        data = np.loadtxt(f, delimiter=",")
        data = data.T

        # get the input and label
        x = data[:, 0]
        y = data[:, 1]

        # reshape to 2D
        x = x.reshape(-1, 1)
        y = y.reshape(-1, 1)

        # combine into a single numpy array
        if i == 0:
            X = x
            Y = y
        else:
            X = np.concatenate((X, x), axis=0)
            Y = np.concatenate((Y, y), axis=0)


    X = torch.tensor(X, dtype=torch.float32)
    X = X.squeeze()
    Y = torch.tensor(Y, dtype=torch.float32)
    Y = Y.squeeze()

    print(f"Data type of X: {X.dtype} | Data type of Y: {Y.dtype}")
    print(f"Size of X: {X.size()} | Size of Y: {Y.size()}")

    # combine into a single dataset
    dataset = torch.utils.data.TensorDataset(X, Y)

    return dataset

In [ ]:
data = load_data()